In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
val1 = torch.FloatTensor([1, 2, 3])

In [3]:
hypothesis = F.softmax(val1, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [4]:
hypothesis.sum()

tensor(1.)

# Low-Level with Cross Entropy Loss

In [8]:
val1 = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(val1, dim=1)
print(hypothesis)

tensor([[0.2225, 0.2279, 0.1137, 0.2130, 0.2229],
        [0.2539, 0.2801, 0.1167, 0.1839, 0.1654],
        [0.2481, 0.2701, 0.1683, 0.1633, 0.1502]], grad_fn=<SoftmaxBackward>)


In [10]:
y = torch.randint(5, (3, )).long()
y

tensor([0, 1, 1])

In [12]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [15]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.3616, grad_fn=<MeanBackward0>)


In [16]:
torch.log(F.softmax(val1, dim=1))

tensor([[-1.5030, -1.4789, -2.1739, -1.5466, -1.5009],
        [-1.3710, -1.2726, -2.1478, -1.6932, -1.7995],
        [-1.3939, -1.3091, -1.7822, -1.8120, -1.8955]], grad_fn=<LogBackward>)

In [17]:
F.log_softmax(val1, dim=1)

tensor([[-1.5030, -1.4789, -2.1739, -1.5466, -1.5009],
        [-1.3710, -1.2726, -2.1478, -1.6932, -1.7995],
        [-1.3939, -1.3091, -1.7822, -1.8120, -1.8955]],
       grad_fn=<LogSoftmaxBackward>)

In [18]:
#PyTorch also has the f.nLL_Loss() function that computes the negative loss likelihood

#low-level
(y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

tensor(1.3616, grad_fn=<MeanBackward0>)

In [19]:
#high-level
F.nll_loss(F.log_softmax(val1, dim=1), y)

tensor(1.3616, grad_fn=<NllLossBackward>)

In [20]:
F.cross_entropy(val1, y)

tensor(1.3616, grad_fn=<NllLossBackward>)

# Training with Low-Level Cross Entropy Loss

In [23]:
x_data = [[1, 2, 1, 1], 
                  [2, 1, 3, 2], 
                  [3, 1, 3, 4], 
                  [4, 1, 5, 5], 
                  [1, 7, 5, 5], 
                  [1, 2, 5, 6], 
                  [1, 6, 6, 6], 
                  [1, 7, 7, 7]]
y_data = [2, 2, 2, 1, 1, 1, 0, 0] 
x_data = torch.FloatTensor(x_data)
y_data = torch.LongTensor(y_data)

In [30]:
#Model initialization
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

#optimizer
optimizer = optim.SGD([W, b], lr=0.1)

for epoch in range(1001):
    hypothesis = F.softmax(x_data.matmul(W) + b, dim=1) 
    y_one_hot = torch.zeros_like(hypothesis) 
    y_one_hot.scatter_(1, y_data.unsqueeze(1), 1) 
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean() 
    optimizer.zero_grad() 
    cost.backward() 
    optimizer.step() 
    if epoch % 100 == 0: 
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, 1000, cost.item())) 

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749
